<a href="https://colab.research.google.com/github/TuckerArrants/moa/blob/main/moa-resnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --q iterative-stratification

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from tensorflow.keras import layers,regularizers,Sequential,Model,backend,callbacks,optimizers,metrics,losses
import tensorflow as tf
from sklearn.metrics import log_loss
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
import sys
import json
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import warnings
warnings.filterwarnings('ignore')

p_min = 0.0005
p_max = 0.9995

In [ ]:
def preprocess(train,test):
    
    og_feats = [_ for _ in [_ for _ in train.columns if _ != 'sig_id'] if _ != 'cp_type']
    train_non_ctl_idx = train.loc[train['cp_type'] != 'ctl_vehicle'].index.to_list()
    train = train.loc[train_non_ctl_idx]
    test_ctl_idx = test.loc[test['cp_type']=='ctl_vehicle'].index.to_list()
    
    train.drop(['sig_id','cp_type'],axis=1, inplace=True)
    test.drop(['sig_id','cp_type'],axis=1, inplace=True)
    
    for df in [train, test]:
        df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})

    scaler = preprocessing.StandardScaler()
    train_ = scaler.fit_transform(train)
    test_ = scaler.transform(test)
    
    train = pd.DataFrame(train_, index=train.index, columns=train.columns)
    test = pd.DataFrame(test_, index=test.index, columns=test.columns)
    
    return train, test, og_feats, train_non_ctl_idx, test_ctl_idx

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))

def variance_threshold_selector(data, threshold=0.8):
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

def fe_stats(train, test, extra=True):
    
    features_g = [col for col in train.columns if 'g-' in col]
    features_c = [col for col in train.columns if 'c-' in col]

    for df in [train, test]:
        df['g_sum'] = df[features_g].sum(axis=1)
        df['g_mean'] = df[features_g].mean(axis=1)
        df['g_std'] = df[features_g].std(axis=1)
        df['g_kurt'] = df[features_g].kurtosis(axis=1)
        df['g_skew'] = df[features_g].skew(axis=1)
        df['g_std'] = df[features_g].std(axis=1)
        df['c_sum'] = df[features_c].sum(axis=1)
        df['c_mean'] = df[features_c].mean(axis=1)
        df['c_std'] = df[features_c].std(axis=1)
        df['c_kurt'] = df[features_c].kurtosis(axis=1)
        df['c_skew'] = df[features_c].skew(axis=1)
        df['c_std'] = df[features_c].std(axis=1)
        df['gc_sum'] = df[features_g + features_c].sum(axis=1)
        df['gc_mean'] = df[features_g + features_c].mean(axis=1)
        df['gc_std'] = df[features_g + features_c].std(axis=1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis=1)
        df['gc_skew'] = df[features_g + features_c].skew(axis=1)
        df['gc_std'] = df[features_g + features_c].std(axis=1)
        
        #these might as well be arbitrary
        if extra:
            df['g_sum-c_sum'] = df['g_sum'] - df['c_sum']
            df['g_mean-c_mean'] = df['g_mean'] - df['c_mean']
            df['g_std-c_std'] = df['g_std'] - df['c_std']
            df['g_kurt-c_kurt'] = df['g_kurt'] - df['c_kurt']
            df['g_skew-c_skew'] = df['g_skew'] - df['c_skew']
            
            df['g_sum*c_sum'] = df['g_sum'] * df['c_sum']
            df['g_mean*c_mean'] = df['g_mean'] * df['c_mean']
            df['g_std*c_std'] = df['g_std'] * df['c_std']
            df['g_kurt*c_kurt'] = df['g_kurt'] * df['c_kurt']
            df['g_skew*c_skew'] = df['g_skew'] * df['c_skew']
            
            df['g_sum/c_sum'] = df['g_sum'] / df['c_sum']
            df['g_mean/c_mean'] = df['g_mean'] / df['c_mean']
            df['g_std/c_std'] = df['g_std'] / df['c_std']
            df['g_kurt/c_kurt'] = df['g_kurt'] / df['c_kurt']
            df['g_skew/c_skew'] = df['g_skew'] / df['c_skew']
        
    return train, test

def fe_pca(train, test, n_components_g=70, n_components_c=10, SEED=123):
    
    features_g = [col for col in train.columns if 'g-' in col]
    features_c = [col for col in train.columns if 'c-' in col]
    
    def create_pca(train, test, features, kind='g', n_components=n_components_g):
        
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis=0)
        pca = PCA(n_components=n_components,  random_state=SEED)
        data = pca.fit_transform(data)
        columns = [f'pca_{kind}{i + 1}' for i in range(n_components)]
        data = pd.DataFrame(data, columns=columns)
        train_ = data.iloc[:train.shape[0]]
        test_ = data.iloc[train.shape[0]:].reset_index(drop=True)
        train = pd.concat([train.reset_index(drop=True),
                           train_.reset_index(drop=True)], axis=1)
        test = pd.concat([test.reset_index(drop=True),
                          test_.reset_index(drop=True)], axis=1)
        return train, test
    
    train, test = create_pca(train, test, features_g, kind='g', n_components=n_components_g)
    train, test = create_pca(train, test, features_c, kind='c', n_components=n_components_c)
    
    return train, test

def mean_log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    metrics = []
    for target in range(206):
        metrics.append(log_loss(y_true[:, target], y_pred[:, target]))
    return np.mean(metrics)

def resnet_block(inputs1, inputs2, dropout, dim1=512, dim2=256):
    block = tf.keras.layers.BatchNormalization()(inputs1)
    block = tf.keras.layers.Dropout(0.2)(block)
    block = tf.keras.layers.Dense(512, "elu")(block)
    block = tf.keras.layers.BatchNormalization()(block)
    block_out = tf.keras.layers.Dense(256, "elu")(block)
    block_out_concat = tf.keras.layers.Concatenate()([inputs2, block])
    return block_out, block_out_concat

def build_resnet2(shape1, shape2, classes, LR=1e-3):  
    
    input_1 = tf.keras.layers.Input(shape=(shape1))
    input_2 = tf.keras.layers.Input(shape=(shape2))
    
    block1_out, block1_out_concat = resnet_block(input_1, input_2, dropout=.2)
    block2_out, _ = resnet_block(block1_out_concat, input_2, dropout=.3)
    
    block_avg = tf.keras.layers.Average()([block1_out, block2_out])
    block_avg = tf.keras.layers.BatchNormalization()(block_avg)
    block_avg = tf.keras.layers.Dense(256, kernel_initializer='lecun_normal', activation='selu')(block_avg)
    block_avg = tf.keras.layers.BatchNormalization()(block_avg)
    block_avg = tf.keras.layers.Dense(classes, kernel_initializer='lecun_normal', activation='selu')(block_avg)
    block_avg = tf.keras.layers.BatchNormalization()(block_avg)
    output = tf.keras.layers.Dense(classes,"sigmoid")(block_avg)

    model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=output)
    adam = tf.optimizers.Adam(learning_rate=LR)
    model.compile(optimizer = adam, 
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.0015), 
                  metrics=tf.keras.metrics.BinaryCrossentropy())
    
    return model

def build_resnet3(shape1, shape2, shape3, classes, LR=1e-3):  
    
    input_1 = tf.keras.layers.Input(shape=(shape1))
    input_2 = tf.keras.layers.Input(shape=(shape2))
    input_3 = tf.keras.layers.Input(shape=(shape3))
    
    block1_out, block1_out_concat = resnet_block(input_1, input_2, dropout=.2)
    block2_out, block2_out_concat = resnet_block(block1_out_concat, input_3, dropout=.3)
    block3_out, _ = resnet_block(block2_out_concat, input_3, dropout=.3)

    block_avg = tf.keras.layers.Average()([block1_out, block2_out, block3_out])
    block_avg = tf.keras.layers.BatchNormalization()(block_avg)
    block_avg = tf.keras.layers.Dense(256, kernel_initializer='lecun_normal', activation='selu')(block_avg)
    block_avg = tf.keras.layers.BatchNormalization()(block_avg)
    block_avg = tf.keras.layers.Dense(classes, kernel_initializer='lecun_normal', activation='selu')(block_avg)
    block_avg = tf.keras.layers.BatchNormalization()(block_avg)
    output = tf.keras.layers.Dense(classes,"sigmoid")(block_avg)

    model = tf.keras.models.Model(inputs=[input_1, input_2, input_3], outputs=output)
    adam = tf.optimizers.Adam(learning_rate=LR)
    model.compile(optimizer = adam, 
                  loss=tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.0015), 
                  metrics=tf.keras.metrics.BinaryCrossentropy())
    
    return model

start_predictors = ["g-0", "g-7", "g-8", "g-10", "g-13", "g-17", "g-20", "g-22", "g-24", "g-26", "g-28", "g-29", "g-30", "g-31", "g-32", "g-34", "g-35", "g-36", "g-37", "g-38",
                    "g-39","g-41", "g-46", "g-48", "g-50", "g-51", "g-52", "g-55", "g-58", "g-59", "g-61", "g-62", "g-63", "g-65", "g-66", "g-67", "g-68", "g-70", "g-72", "g-74", 
                    "g-75", "g-79", "g-83", "g-84", "g-85", "g-86", "g-90", "g-91", "g-94", "g-95", "g-96", "g-97", "g-98", "g-100", "g-102", "g-105", "g-106", "g-112", "g-113", 
                    "g-114", "g-116", "g-121", "g-123", "g-126", "g-128", "g-131", "g-132", "g-134", "g-135", "g-138", "g-139", "g-140", "g-142", "g-144", "g-145", "g-146", 
                    "g-147", "g-148", "g-152", "g-155", "g-157", "g-158", "g-160", "g-163", "g-164", "g-165", "g-170", "g-173", "g-174", "g-175", "g-177", "g-178", "g-181", 
                    "g-183", "g-185", "g-186", "g-189", "g-192", "g-194", "g-195", "g-196", "g-197", "g-199", "g-201", "g-202", "g-206", "g-208", "g-210", "g-213", "g-214", 
                    "g-215", "g-220", "g-226", "g-228", "g-229", "g-235", "g-238", "g-241", "g-242", "g-243", "g-244", "g-245", "g-248", "g-250", "g-251", "g-254", "g-257", 
                    "g-259", "g-261", "g-266", "g-270", "g-271", "g-272", "g-275", "g-278", "g-282", "g-287", "g-288", "g-289", "g-291", "g-293", "g-294", "g-297", "g-298",
                    "g-301", "g-303", "g-304", "g-306", "g-308", "g-309", "g-310", "g-311", "g-314", "g-315", "g-316", "g-317", "g-320", "g-321", "g-322", "g-327", "g-328", 
                    "g-329", "g-332", "g-334", "g-335", "g-336", "g-337", "g-339", "g-342", "g-344", "g-349", "g-350", "g-351", "g-353", "g-354", "g-355", "g-357", "g-359", 
                    "g-360", "g-364", "g-365", "g-366", "g-367", "g-368", "g-369", "g-374", "g-375", "g-377", "g-379", "g-385", "g-386", "g-390", "g-392", "g-393", "g-400", 
                    "g-402", "g-406", "g-407", "g-409", "g-410", "g-411", "g-414", "g-417", "g-418", "g-421", "g-423", "g-424", "g-427", "g-429", "g-431", "g-432", "g-433", 
                    "g-434", "g-437", "g-439", "g-440", "g-443", "g-449", "g-458", "g-459", "g-460", "g-461", "g-464", "g-467", "g-468", "g-470", "g-473", "g-477", "g-478", 
                    "g-479", "g-484", "g-485", "g-486", "g-488", "g-489", "g-491", "g-494", "g-496", "g-498", "g-500", "g-503", "g-504", "g-506", "g-508", "g-509", "g-512", 
                    "g-522", "g-529", "g-531", "g-534", "g-539", "g-541", "g-546", "g-551", "g-553", "g-554", "g-559", "g-561", "g-562", "g-565", "g-568", "g-569", "g-574", 
                    "g-577", "g-578", "g-586", "g-588", "g-590", "g-594", "g-595", "g-596", "g-597", "g-599", "g-600", "g-603", "g-607", "g-615", "g-618", "g-619", "g-620", 
                    "g-625", "g-628", "g-629", "g-632", "g-634", "g-635", "g-636", "g-638", "g-639", "g-641", "g-643", "g-644", "g-645", "g-646", "g-647", "g-648", "g-663", 
                    "g-664", "g-665", "g-668", "g-669", "g-670", "g-671", "g-672", "g-673", "g-674", "g-677", "g-678", "g-680", "g-683", "g-689", "g-691", "g-693", "g-695", 
                    "g-701", "g-702", "g-703", "g-704", "g-705", "g-706", "g-708", "g-711", "g-712", "g-720", "g-721", "g-723", "g-724", "g-726", "g-728", "g-731", "g-733", 
                    "g-738", "g-739", "g-742", "g-743", "g-744", "g-745", "g-749", "g-750", "g-752", "g-760", "g-761", "g-764", "g-766", "g-768", "g-770", "g-771", "c-0", 
                    "c-1", "c-2", "c-3", "c-4", "c-5", "c-6", "c-7", "c-8", "c-9", "c-10", "c-11", "c-12", "c-13", "c-14", "c-15", "c-16", "c-17", "c-18", "c-19", "c-20", 
                    "c-21", "c-22", "c-23", "c-24", "c-25", "c-26", "c-27", "c-28", "c-29", "c-30", "c-31", "c-32", "c-33", "c-34", "c-35", "c-36", "c-37", "c-38", "c-39", 
                    "c-40", "c-41", "c-42", "c-43", "c-44", "c-45", "c-46", "c-47", "c-48", "c-49", "c-50", "c-51", "c-52", "c-53", "c-54", "c-55", "c-56", "c-57", "c-58", 
                    "c-59", "c-60", "c-61", "c-62", "c-63", "c-64", "c-65", "c-66", "c-67", "c-68", "c-69", "c-70", "c-71", "c-72", "c-73", "c-74", "c-75", "c-76", "c-77", 
                    "c-78", "c-79", "c-80", "c-81", "c-82", "c-83", "c-84", "c-85", "c-86", "c-87", "c-88", "c-89", "c-90", "c-91", "c-92", "c-93", "c-94", "c-95", "c-96", 
                    "c-97", "c-98", "c-99"]

# Model

In [ ]:
#these are dummy values
model2 = build_resnet2(1000, 700, 206)
model3 = build_resnet3(1000, 800, 700, 206)
tf.keras.utils.plot_model(model2,show_shapes=True)

In [ ]:
tf.keras.utils.plot_model(model3,show_shapes=True)

# Training

In [ ]:
TRAIN_BOTH = True
TRANSFER = False
VAR_THRESH = False
ADD_PCA = False
PCA_G_FEATS = 100
PCA_C_FEATS = 10

SEEDS = [34, 35, 36]
FOLDS = 8
EPOCHS = 10
BATCH_SIZE = 128
VERBOSE = 2

#nonscored training params
EPOCH_LIST = [5, 3, 3]
BATCH_SIZE_LIST = [64, 128, 256]

In [ ]:
test = pd.read_csv('test_features.csv')
oof_preds2 = []
oof_preds3 = []
seed_losses2 = []
seed_losses3 = []
test_pred = np.zeros((test.shape[0], 206)) 

for seed in SEEDS:

    print("#"*25)
    print(f"Training seed {seed}...")
    print("#"*25)
    print('')

###################################################
### Get data
###################################################

    train = pd.read_csv('train_features.csv')
    train_targets = pd.read_csv('train_targets_scored.csv')
    train_targets_ns = pd.read_csv('train_targets_nonscored.csv')
    test = pd.read_csv('test_features.csv')
    train_targets.drop('sig_id', axis=1, inplace=True)
    train_targets_ns.drop('sig_id', axis=1, inplace=True)

###################################################
### Processes data
###################################################

    train, test = fe_stats(train, test)
    train, test, og_feats, train_non_ctl_idx, test_ctl_idx = preprocess(train, test)
    train_targets_ns = train_targets_ns.iloc[train_non_ctl_idx]
    train_targets = train_targets.iloc[train_non_ctl_idx]

    if ADD_PCA:
        train, test = fe_pca(train, test, n_components_g=PCA_G_FEATS, n_components_c=PCA_C_FEATS, SEED=seed)

    if VAR_THRESH:
        total = pd.concat([train, test], ignore_index=True)
        total = variance_threshold_selector(total, .8)
        train = total[:len(train)]
        test = total[len(train):]

    all_feats = train.columns 
    oof_pred2 = np.zeros((train.shape[0], 206))
    oof_pred3 = np.zeros((train.shape[0], 206))
    
    skf = MultilabelStratifiedKFold(n_splits=FOLDS, random_state=seed)

    for f, (train_index, val_index) in enumerate(skf.split(train.values, train_targets.values)):

        print(f"Training fold {f}...")
        print('')

        x_train, x_val = train[all_feats].values[train_index], train[all_feats].values[val_index]
        y_train, y_val = train_targets.values[train_index], train_targets.values[val_index]
        x_train_, x_val_ = train[og_feats].values[train_index], train[og_feats].values[val_index]
        x_train__, x_val__ = train[start_predictors].values[train_index], train[start_predictors].values[val_index]

###################################################
### Build model
###################################################

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy',
                                                          mode='min',
                                                          patience=10,
                                                          restore_best_weights=False,
                                                          verbose=VERBOSE)

        reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_binary_crossentropy',
                                                              mode='min',
                                                              factor=0.1,
                                                              patience=4,
                                                              verbose=VERBOSE)

        sv2 = tf.keras.callbacks.ModelCheckpoint(f'resnet2_{f}_{seed}.h5',
                                                monitor='val_binary_crossentropy',
                                                verbose=VERBOSE,
                                                save_best_only=True,
                                                save_weights_only=True)

        sv3 = tf.keras.callbacks.ModelCheckpoint(f'resnet3_{f}_{seed}.h5',
                                                monitor='val_binary_crossentropy',
                                                verbose=VERBOSE,
                                                save_best_only=True,
                                                save_weights_only=True)

        sv2_ns = tf.keras.callbacks.ModelCheckpoint(f'resnet2_{f}_{seed}_ns.h5',
                                                monitor='val_binary_crossentropy',
                                                verbose=VERBOSE,
                                                save_best_only=True,
                                                save_weights_only=True)

        sv3_ns = tf.keras.callbacks.ModelCheckpoint(f'resnet3_{f}_{seed}_ns.h5',
                                                monitor='val_binary_crossentropy',
                                                verbose=VERBOSE,
                                                save_best_only=True,
                                                save_weights_only=True)

        resnet2 = build_resnet2(len(all_feats), len(start_predictors), train_targets.shape[1])
        resnet3 = build_resnet3(len(all_feats), len(og_feats), len(start_predictors), train_targets.shape[1])

###################################################
### Train model and infer
###################################################

        if TRANSFER:

            y_train_ns, y_val_ns = train_targets_ns.values[train_index], train_targets_ns.values[val_index]

            resnet2_ns = build_resnet2(len(all_feats), len(start_predictors), train_targets_ns.shape[1])
            resnet3_ns = build_resnet3(len(all_feats), len(og_feats), len(start_predictors), train_targets_ns.shape[1])
            print('--'*25)
            print('Training ResNet with 2 inputs on nonscored targets...')
            print('--'*25)
            for epoch, batch_size in zip(EPOCH_LIST, BATCH_SIZE_LIST):
                resnet2_ns.fit([x_train, x_train__], y_train_ns,
                            validation_data = ([x_val, x_val__], y_val_ns),
                            epochs=epoch, 
                            batch_size=batch_size,
                            callbacks=[sv2_ns, reduce_lr_loss],
                            verbose=VERBOSE)
                #resnet2_ns.fit([x_train, x_train], y_train_ns,
                            #validation_data = ([x_val, x_val], y_val_ns),
                            #epochs=epoch, 
                            #batch_size=batch_size,
                            #callbacks=[sv2_ns, reduce_lr_loss],
                            #verbose=VERBOSE)
            resnet2_ns.load_weights(f'resnet2_{f}_{seed}_ns.h5')

            if TRAIN_BOTH:
                print('')
                print('--'*25)
                print('Training ResNet with 3 inputs on nonscored targets...')
                print('--'*25)
                print('')
                for epoch, batch_size in zip(EPOCH_LIST, BATCH_SIZE_LIST):
                    resnet3_ns.fit([x_train, x_train_, x_train__], y_train_ns,
                                validation_data = ([x_val, x_val_, x_val__], y_val_ns),
                                epochs=epoch, 
                                batch_size=batch_size,
                                callbacks=[sv3_ns, reduce_lr_loss],
                                verbose=VERBOSE)
                    #resnet3_ns.fit([x_train, x_train, x_train], y_train_ns,
                                #validation_data = ([x_val, x_val, x_val], y_val_ns),
                                #epochs=epoch, 
                                #batch_size=batch_size,
                                #callbacks=[sv3_ns, reduce_lr_loss],
                                #verbose=VERBOSE)
                print('')
                resnet3_ns.load_weights(f'resnet3_{f}_{seed}_ns.h5')

        print('--'*25)
        print('Training ResNet with 2 inputs after transfering weights...' if TRANSFER else 'Training ResNet with 2 inputs...')
        print('--'*25)
        print('')

        if TRANSFER:
            for i, layer in enumerate(resnet2.layers[:-1]):
                resnet2.layers[i] = resnet2_ns.layers[i]

        resnet2.fit([x_train, x_train__], y_train,
                    validation_data = ([x_val, x_val__], y_val),
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE,
                    callbacks=[sv2, reduce_lr_loss],
                    verbose=VERBOSE)
        #resnet2.fit([x_train, x_train], y_train,
                    #validation_data = ([x_val, x_val], y_val),
                    #epochs=EPOCHS, 
                    #batch_size=BATCH_SIZE,
                    #callbacks=[sv2, reduce_lr_loss],
                    #verbose=VERBOSE)

        print('')
        resnet2.load_weights(f'resnet2_{f}_{seed}.h5')
        oof_pred2[val_index] = resnet2.predict([x_val, x_val__])
        test_pred += resnet2.predict([test[all_feats].values, test[start_predictors].values]) / (FOLDS * len(SEEDS))

        if TRAIN_BOTH:
            print('--'*25)
            print('Training ResNet with 3 inputs after transfering weights...' if TRANSFER else 'Training ResNet with 3 inputs...') 
            print('--'*25)

            if TRANSFER:
                for i, layer in enumerate(resnet2.layers[:-1]):
                    resnet2.layers[i] = resnet2_ns.layers[i]

            resnet3.fit([x_train, x_train_, x_train__], y_train,
                        validation_data = ([x_val, x_val_, x_val__], y_val),
                        epochs=EPOCHS, 
                        batch_size=BATCH_SIZE,
                        callbacks=[sv3, reduce_lr_loss],
                        verbose=VERBOSE)
            #resnet3.fit([x_train, x_train, x_train], y_train,
                        #validation_data = ([x_val, x_val, x_val], y_val),
                        #epochs=EPOCHS, 
                        #batch_size=BATCH_SIZE,
                        #callbacks=[sv3, reduce_lr_loss],
                        #verbose=VERBOSE)
            print('')   

            resnet3.load_weights(f'resnet3_{f}_{seed}.h5')
            oof_pred3[val_index] = resnet3.predict([x_val, x_val_, x_val__])
            test_pred += resnet3.predict([test[all_feats].values, test[og_feats].values, test[start_predictors].values]) / (FOLDS * len(SEEDS))

    oof_preds2.append(oof_pred2)
    seed_loss2 = mean_log_loss(train_targets.values, oof_pred2)
    seed_losses2.append(seed_loss2)
    print(f"ResNet with 2 inputs {seed} mean log loss: {seed_loss2}")
    print('')

    if TRAIN_BOTH:
        oof_preds3.append(oof_pred3)
        seed_loss3 = mean_log_loss(train_targets.values, oof_pred3)
        seed_losses3.append(seed_loss3)
        print(f"ResNet with 3 inputs {seed} mean log loss: {seed_loss3}")
        print('')

In [ ]:
print('ResNet with 2 inputs results:')
for i, score in enumerate(seed_losses2):
    print(f"Seed {SEEDS[i]} score: {score}")

oof_preds2_ = np.zeros((train.shape[0], 206)) 
for _ in oof_preds2:
    oof_preds2_ += _
oof_preds2_ *= 1/len(SEEDS)

overall_loss2 = mean_log_loss(train_targets.values, oof_preds2_)

print(f"Overall loss: {overall_loss2}")
print('')

if TRAIN_BOTH:
    print('ResNet with 3 inputs results:')
    for i, score in enumerate(seed_losses3):
        print(f"Seed {SEEDS[i]} score: {score}")

    oof_preds3_ = np.zeros((train.shape[0], 206)) 
    for _ in oof_preds3:
        oof_preds3_ += _
    oof_preds3_ *= 1/len(SEEDS)

    overall_loss3 = mean_log_loss(train_targets.values, oof_preds3_)

    print(f"Overall loss: {overall_loss3}")

    overall_loss = mean_log_loss(train_targets.values, (oof_preds2_ + oof_preds3_) / 2)
    print(f'Blended loss : {overall_loss}')

In [ ]:
if TRAIN_BOTH:
  test_pred *= 1/2
sub = pd.read_csv('sample_submission.csv')
sub.iloc[:,1:] = np.clip(test_pred,p_min,p_max)
sub.iloc[test_ctl_idx,1:] = 0
sub.to_csv('submission.csv', index=False)
sub